![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# FOREX Strategy using Corrective Artificial Intelligence (CAI)

This notebook connects to PredictNow, trains a model, and generates predictions.

The model hypothesis is that USD will rise against the EUR during EUR business hours and fall during the USD business hours. This is called the time of the day effect and seen due to HF OF and returns (https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2099321).

### Connect to PredictNow

In [ ]:
from AlgorithmImports import *
from QuantConnect.PredictNowNET import PredictNowClient
from QuantConnect.PredictNowNET.Models import *
from datetime import datetime, time
from io import StringIO
import pandas as pd

qb = QuantBook()
client = PredictNowClient("jared@quantconnect.com", "jared_broad")
client.connected

### Prepare the Data
In this notebook, we will create a strategy that short EURUSD when Europe is open and long when Europe is closed and US is open. We will aggregate the daily return of this static strategy that is activate everyday, and use CAI to predict if the strategy is profitable for a given date. We will follow this On and Off signal to create a dynamic strategy and benchmark its performance.

In [ ]:
# load minute bar data of EURUSD
symbol = qb.add_forex("EURUSD").symbol
df_price = qb.History(symbol, datetime(2020,1,1), datetime(2021,1,1)).loc[symbol]

# resample to hourly returns
minute_returns = df_price["close"].pct_change()
hourly_returns = (minute_returns + 1).resample('H').prod() - 1
df_hourly_returns = hourly_returns.to_frame()
df_hourly_returns['time'] = df_hourly_returns.index.time

# generate buy and sell signals and get strategy returns
# Sell EUR.USD when Europe is open
sell_eur = ((df_hourly_returns['time'] > time(3)) & (df_hourly_returns['time'] < time(9)))

# Buy EUR.USD when Europe is closed and US is open
buy_eur = ((df_hourly_returns['time'] > time(11)) & (df_hourly_returns['time'] < time(15)))

# signals as 1 and -1
ones = pd.DataFrame(1, index=df_hourly_returns.index, columns=['signals'])
minus_ones = pd.DataFrame(-1, index=df_hourly_returns.index, columns=['signals'])
signals = minus_ones.where(sell_eur, ones.where(buy_eur, 0))

# strategy returns
strategy_returns = df_hourly_returns['close'] * signals['signals']
strategy_returns = (strategy_returns + 1).resample('D').prod() - 1
df_strategy_returns = strategy_returns.to_frame().ffill()

### Save the Data
We will label the data and save it to disk (ObjectStore) with the model name. This file will be uploaded to PredictNow.

In [ ]:
# Define the model name and data lable
model_name = "fx-time-of-day"
label =  "strategy_ret"

# Label the data and save it to the object store
df_strategy_returns = df_strategy_returns.rename(columns={df_strategy_returns.columns.to_list()[0]: label})
parquet_path = qb.object_store.get_file_path(f'{model_name}.parquet')
df_strategy_returns.to_parquet(parquet_path)

### Create the Model
Create the model by sending the parameters to PredictNow

In [ ]:
model_parameters = ModelParameters(
    mode=Mode.TRAIN, 
    type=ModelType.CLASSIFICATION, 
    feature_selection=FeatureSelection.SHAP, 
    analysis=Analysis.SMALL, 
    boost=Boost.GBDT, 
    testsize=42.0,
    timeseries=False,
    probability_calibration=False,    # True if we want to refine your probability
    exploratory_data_analysis=False,  # True if we want to use exploratory data analysis
    weights="no")                     # yes, no, custom

create_model_result = client.create_model(model_name, model_parameters)
str(create_model_result)

### Train the Model
Provide the path to the data, and its label.
This task may take several minutes.

In [ ]:
train_request_result = client.train(model_name, parquet_path, label)
str(train_request_result)

### Get the training result
The training results include dataframes with eprformance metrics and predicted probability and labels.  

In [ ]:
training_result = client.get_training_result(model_name)
str(training_result)

# Predicted probability (float between 0 and 1) for validation/training data set
# the last column notes the probability that it's a "1", i.e. positive return
predicted_prob_cv = pd.read_json(StringIO(training_result.predicted_prob_cv))
print("predicted_prob_cv")
print(predicted_prob_cv)

# Predicted probability (float between 0 and 1) for the testing data set
predicted_prob_test = pd.read_json(StringIO(training_result.predicted_prob_test))
print("predicted_prob_test")
print(predicted_prob_test)

# Predicted label, 0 or 1, for validation/training data set. Classified as class 1 if probability > 0.5
predicted_targets_cv = pd.read_json(StringIO(training_result.predicted_targets_cv))
print("predicted_targets_cv")
print(predicted_targets_cv)

# Predicted label, 0 or 1, for testing data set. Classified as class 1 if probability > 0.5
predicted_targets_test = pd.read_json(StringIO(training_result.predicted_targets_test))
print("predicted_targets_test")
print(predicted_targets_test)

# Feature importance score, shows what features are being used in the prediction
# More helpful when you include your features
# and only works when you set feature_selection to FeatureSelection.SHAP or FeatureSelection.CMDA
if training_result.feature_importance:
    feature_importance = pd.read_json(StringIO(training_result.feature_importance))
    print("feature_importance")
    print(feature_importance)

# Performance metrics in terms of accuracies
performance_metrics = pd.read_json(StringIO(training_result.performance_metrics))
print("performance_metrics")
print(performance_metrics)

### Start Predicting with the Trained Model

In [ ]:
predict_result = client.predict(model_name, parquet_path, exploratory_data_analysis=False, probability_calibration=False)
str(predict_result)